In [ ]:
# 필요한 라이브러리 불러오기
from google.colab import files
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

# 파일 업로드
uploaded = files.upload()

# 업로드된 파일 읽기
file_path = list(uploaded.keys())[0]  # 업로드한 파일 이름 가져오기
data = pd.read_csv(file_path, sep="\t")

# 데이터 필터링: 연령(0), 지역(1), clean(2)
filtered_data = data[(data["연령"] == 1) | (data["지역"] == 1) | (data["clean"] == 1)].copy()
filtered_data['label'] = filtered_data.apply(
    lambda row: 0 if row['연령'] == 1 else (1 if row['지역'] == 1 else 2), axis=1
)
filtered_data = filtered_data[['문장', 'label']]

# 데이터셋 나누기
train_data, test_data = train_test_split(filtered_data, test_size=0.2, random_state=42)

# 토크나이저 불러오기
MODEL_NAME = "beomi/KcELECTRA-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# 데이터 토큰화 함수
def tokenize_data(data):
    return tokenizer(
        list(data["문장"]),
        return_tensors="pt",
        max_length=128,
        padding=True,
        truncation=True,
        add_special_tokens=True,
    )

tokenized_train_sentences = tokenize_data(train_data)
tokenized_test_sentences = tokenize_data(test_data)

# 커스텀 데이터셋 클래스 정의
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# 데이터셋 준비
train_labels = train_data["label"].values
test_labels = test_data["label"].values
train_dataset = CustomDataset(tokenized_train_sentences, train_labels)
test_dataset = CustomDataset(tokenized_test_sentences, test_labels)

# 모델 불러오기
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=3)  # 3개 클래스
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 학습 파라미터 설정
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",
    report_to="none"  # W&B 비활성화
)

# 평가지표 함수 정의
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# Trainer 정의
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

# 모델 학습
trainer.train()

# 평가
evaluation_results = trainer.evaluate(eval_dataset=test_dataset)
print(evaluation_results)

# 테스트용 문장 예측 함수
def sentence_predict(sent):
    model.eval()
    tokenized_sent = tokenizer(
        sent,
        return_tensors="pt",
        truncation=True,
        add_special_tokens=True,
        max_length=128
    ).to(device)
    with torch.no_grad():
        outputs = model(**tokenized_sent)
    logits = outputs.logits
    result = logits.argmax(-1).item()
    # 결과값 변환
    if result == 0:
        return "연령혐오 악성댓글"
    elif result == 1:
        return "지역혐오 악성댓글"
    else:
        return "정상댓글"

# 인터랙티브 테스트
while True:
    sentence = input("댓글을 입력해주세요 (종료: 0): ")
    if sentence == "0":
        break
    print(sentence_predict(sentence))
    print()


Saving unsmile_train_v1.0.tsv to unsmile_train_v1.0 (4).tsv


Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-8-921b9fe60be6>:51: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.223800,0.290716,0.930103,0.930268,0.934897,0.930103
2,0.135200,0.239846,0.958062,0.957942,0.957967,0.958062
3,0.122800,0.182163,0.955266,0.954852,0.955125,0.955266
4,0.139500,0.203249,0.965517,0.965440,0.965463,0.965517
5,0.002500,0.217689,0.963653,0.963668,0.963733,0.963653


<ipython-input-8-921b9fe60be6>:51: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-8-921b9fe60be6>:51: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-8-921b9fe60be6>:51: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-8-921b9fe60be6>:51: UserWarning: To copy construct from a 

{'eval_loss': 0.21768918633460999, 'eval_accuracy': 0.9636533084808947, 'eval_f1': 0.963668093790681, 'eval_precision': 0.963733464773702, 'eval_recall': 0.9636533084808947, 'eval_runtime': 5.5444, 'eval_samples_per_second': 193.53, 'eval_steps_per_second': 12.265, 'epoch': 5.0}
댓글을 입력해주세요 (종료: 0): 잼민아. 너는 부모님 등골브레이커 초딩이다. 정신차려라
연령혐오 악성댓글

댓글을 입력해주세요 (종료: 0): 경상도 너네들 때문에 나라가 망했다.
지역혐오 악성댓글

댓글을 입력해주세요 (종료: 0): 세계 평화. 모두를 사랑합시다.
정상댓글

댓글을 입력해주세요 (종료: 0): 성균관대학교 사랑합니다. 이 시대 최고의 대학교
정상댓글

댓글을 입력해주세요 (종료: 0): 이렇게 보이스피싱이 활개치는데 사회적으로 불신을 조장하는 이거 어쩔건데
정상댓글

댓글을 입력해주세요 (종료: 0): 여자들은 집안일이나 해. 밖에 나와서 설치지 말고
정상댓글

댓글을 입력해주세요 (종료: 0): 늙었으면 집에 박혀있어라 ㅠ 틀니 압수
연령혐오 악성댓글

댓글을 입력해주세요 (종료: 0): 0
